# Affine alignment algorithm

In [5]:
def affine_gap_alignment(seq1, seq2,  weight_match, weight_mismatch, sigma_penalty, epsilon_penalty):
    m = len(seq1)
    n = len(seq2)

    L = [[0 for j in range(m + 1)] for i in range(n + 1)]  # second arg T if is a gap to here
    L_coming_from = [[0] * (m + 1) for i in range(n + 1)]
    L[0][1] = -sigma_penalty
    L[1][0] = -sigma_penalty

    for i in range(2, n + 1):
        L[i][0] = L[i - 1][0] - epsilon_penalty
    for j in range(2, m + 1):
        L[0][j] = L[0][j - 1] - epsilon_penalty

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            match = L[i - 1][j - 1] + (weight_match if seq1[j - 1] ==  seq2[i - 1] else weight_mismatch)
            if match > L[i][j]:
                L[i][j] = match
                L_coming_from[i][j] = 0

            for k in range(1, i + 1):
                new_match = L[i - k][j] - sigma_penalty - epsilon_penalty * (k - 1)
                if new_match > L[i][j]:
                    L[i][j] = new_match
                    L_coming_from[i][j] = -k 

            for k in range(1, j + 1):
                new_match = L[i][j - k] - sigma_penalty - epsilon_penalty * (k - 1)
                if new_match > L[i][j]:
                    L[i][j] = new_match
                    L_coming_from[i][j] = k

    score = L[n][m]

    res_seq1, res_seq2 = '', ''
    i = n
    j = m
    while i > 0 and j > 0:
        if L_coming_from[i][j] < 0:
            k = -L_coming_from[i][j]
            i -= k
            res_seq2 = seq2[i:i + k] + res_seq2
            res_seq1 = '-' * k + res_seq1
        elif L_coming_from[i][j] > 0:
            k = L_coming_from[i][j]
            j -= k
            res_seq1 = seq1[j: j + k] + res_seq1
            res_seq2 = '-' * k + res_seq2
        else:
            i -= 1
            j -= 1
            res_seq1 = seq1[j] + res_seq1
            res_seq2 = seq2[i] + res_seq2
    if i > 0:
        res_seq2 = seq2[:i] + res_seq2
        res_seq1 = "-" * i + res_seq1
    elif j > 0:
        res_seq1 = seq1[:j] + res_seq1
        res_seq2 = "-" * j + res_seq2
    return str(score) + '\n' + res_seq1 + '\n' + res_seq2

# Testing the algo.
*Changing the sigma penalty to a very big one makes the alignment less sparse.*

In [6]:
seq1 = 'PRTEINS'
seq2 = 'PRTWPSEIN'
print(affine_gap_alignment(seq1, seq2, 10, -5, 20, 1))

18
PRT---EINS
PRTWPSEIN-


In [15]:
seq1 = 'PRTEINS'
seq2 = 'PRTWPSEIN'
print(affine_gap_alignment(seq1, seq2, 10, -5, 100, 1))

0
--PRTEINS
PRTWPSEIN
